*In this tutorial we will investigate two seperate companies and check if they are connected.*

There are instances where we may want to see if two companies are connected. We can do this by simply building a network for each company and comparing them to see if there are any common officers, addresses or companies.

Lets test this approach with two example companies, Zahawi & Zahawi Ltd (07285998) and Gorgeous Services Limited (05714521):

In [8]:
import sugartrail
import pandas as pd
sugartrail.api.basic_auth.username = ""

Create one network for Zahawi & Zahawi including some limits to reduce the number of possibly irrelevant connections:

In [2]:
zahawi_connections = sugartrail.base.Network(company_id='07285998')
zahawi_connections.hop.officer_appointments_maxsize = 20
zahawi_connections.hop.officers_at_address_maxsize = 20
zahawi_connections.hop.companies_at_address_maxsize = 20

Create a second network for Gorgeous Services:

In [3]:
gorgeous_connections = sugartrail.base.Network(company_id='05714521')
gorgeous_connections.hop.officer_appointments_maxsize = 20
gorgeous_connections.hop.officers_at_address_maxsize = 20
gorgeous_connections.hop.companies_at_address_maxsize = 20

We can now pass both networks to the `find_network_connections` method which returns any connections found between two networks. The method accepts two networks as input and an optional `max_depth` value (defaults to 5) which sets the maximum depth of network we will build for both. `find_network_connections` builds each network up to the `max_depth` value and completes when connections are found or the `max_depth` is reached.

In [4]:
connections = sugartrail.processing.find_network_connections(zahawi_connections, gorgeous_connections)

1/5 hops completed.
2/5 hops completed.
3/5 hops completed.
Found connection(s)!


Looks like a connection was found. We can see by the long string of characters that its an officer ID:

In [5]:
connections

['g8BmvnpH8blqT87i93sgJeowx7I']

We can now trace the path from Zahawi & Zahawi to this connection:

In [6]:
pd.DataFrame(zahawi_connections.find_path('g8BmvnpH8blqT87i93sgJeowx7I'))

,title,depth,node_type,id,link_type,link,node_index
0,ZAHAWI & ZAHAWI LTD,0,Company,07285998,,,a
1,Nadhim ZAHAWI,1,Person,tKup8kXPh3-jx_5Bs-BkF5XCyPM,Officer,a,b
2,YOUGOV PLC,2,Company,03607311,Appointment,b,c
3,Benjamin William ELLIOT,3,Person,g8BmvnpH8blqT87i93sgJeowx7I,Officer,c,d


... and the path from Gorgeous Connections to the connection:

In [7]:
pd.DataFrame(gorgeous_connections.find_path('g8BmvnpH8blqT87i93sgJeowx7I'))

,title,depth,node_type,id,link_type,link,node_index
0,GORGEOUS SERVICES LIMITED,0,Company,05714521,,,a
1,Benjamin William ELLIOT,1,Person,g8BmvnpH8blqT87i93sgJeowx7I,Officer,a,b


Reading both paths tells us how Zahawi & Zahawi connect to Gorgeous Connections. Zahawi & Zahawi has Nadhim Zahawi as an officer who has YOUGOV PLC as an appointment which has Benjamin Elliot as an officer who is also an officer of Gorgeous Services Limited.